<a href="https://colab.research.google.com/github/veronica1908/MARKETING_E2_A_III/blob/main/b_exploracion_limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BIBLIOTECAS Y LIBRERÍAS

In [1]:
# Instalar bibliotecas necesarias
!pip install sqlalchemy
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 8.9 MB/s eta 0:00:00


In [2]:
# Importar bibliotecas
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import plotly.express as px
from sqlalchemy import create_engine

#VERIFICACIÓN DE LA BASE DE DATOS EN SQL

##Ver las tablas de la base de datos

In [8]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

# Crear un cursor
cur = conn.cursor()

# Obtener una lista de tablas en la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Imprimir los nombres de las tablas para verificar la información
for table in tables:
    print(table[0])


ratings
movies


##Ver las primeras filas de las tablas

In [9]:
# Leer las primeras filas de la tabla 'ratings'
df_ratings = pd.read_sql_query("SELECT * FROM ratings LIMIT 5;", conn)
print("Primeras filas de la tabla 'ratings':")
print(df_ratings)

# Leer las primeras filas de la tabla 'movies'
df_movies = pd.read_sql_query("SELECT * FROM movies LIMIT 5;", conn)
print("\nPrimeras filas de la tabla 'movies':")
print(df_movies)


Primeras filas de la tabla 'ratings':
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Primeras filas de la tabla 'movies':
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


##Ver el tipo de datos de cada tabla

In [11]:
# Obtener información sobre las columnas de la tabla 'ratings'
cur.execute("PRAGMA table_info(ratings)")
columns_ratings = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'ratings'
print("Tipos de datos de las columnas en la tabla 'ratings':")
for column in columns_ratings:
    print(column[1] + ": " + column[2])

# Obtener información sobre las columnas de la tabla 'movies'
cur.execute("PRAGMA table_info(movies)")
columns_movies = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'movies'
print("\nTipos de datos de las columnas en la tabla 'movies':")
for column in columns_movies:
    print(column[1] + ": " + column[2])

Tipos de datos de las columnas en la tabla 'ratings':
userId: INTEGER
movieId: INTEGER
rating: REAL
timestamp: INTEGER

Tipos de datos de las columnas en la tabla 'movies':
movieId: INTEGER
title: TEXT
genres: TEXT


#TRAER TABLAS A PYTHON

##Tabla ratings

In [13]:
# Consulta SQL para seleccionar todos los datos de la tabla especificada
query = f"SELECT * FROM {'ratings'}"

# Leer los datos de la tabla en un DataFrame de Pandas
dfr = pd.read_sql_query(query, conn)

# Mostrar las primeras filas del DataFrame
print("Primeras filas de la tabla '{}':".format('ratings'))
print(dfr.head())

Primeras filas de la tabla 'ratings':
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


##Tabla movies

In [14]:
# Consulta SQL para seleccionar todos los datos de la tabla especificada
query = f"SELECT * FROM {'movies'}"

# Leer los datos de la tabla en un DataFrame de Pandas
dfm = pd.read_sql_query(query, conn)

# Mostrar las primeras filas del DataFrame
print("Primeras filas de la tabla '{}':".format('movies'))
print(dfm.head())

Primeras filas de la tabla 'movies':
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


#EXPLORACIÓN INICIAL

In [15]:
# Identificar los campos de cruce
# Suponiendo que 'movieId' es el campo de cruce en ambas tablas
print("\nCampos de cruce:")
print("Tabla 'ratings':", dfr.columns)
print("Tabla 'movies':", dfm.columns)

# Verificar que los campos de cruce estén en el mismo formato
# Comprobación del formato de 'movieId' en ambas tablas
print("\nFormato de 'movieId':")
print("Tabla 'ratings':", dfr['movieId'].dtype)
print("Tabla 'movies':", dfm['movieId'].dtype)

# Verificar duplicados en ambas tablas
duplicates_ratings = dfr.duplicated().sum()
duplicates_movies = dfm.duplicated().sum()

print("\nCantidad de duplicados en la tabla 'ratings':", duplicates_ratings)
print("Cantidad de duplicados en la tabla 'movies':", duplicates_movies)


Campos de cruce:
Tabla 'ratings': Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Tabla 'movies': Index(['movieId', 'title', 'genres'], dtype='object')

Formato de 'movieId':
Tabla 'ratings': int64
Tabla 'movies': int64

Cantidad de duplicados en la tabla 'ratings': 0
Cantidad de duplicados en la tabla 'movies': 0


##Descripción base ratings

##Distribución de las calificaciones

In [16]:
# Consulta SQL para calcular la distribución de calificaciones
query = """
        SELECT
            rating,
            COUNT(*) AS conteo
        FROM
            ratings
        GROUP BY
            rating
        ORDER BY
            rating ASC
        """

# Ejecutar la consulta y cargar los resultados en un DataFrame
dfr2 = pd.read_sql_query(query, conn)

# Imprimir el DataFrame resultante
print(dfr2)

   rating  conteo
0     0.5    1370
1     1.0    2811
2     1.5    1791
3     2.0    7551
4     2.5    5550
5     3.0   20047
6     3.5   13136
7     4.0   26818
8     4.5    8551
9     5.0   13211


In [17]:
# Crear el objeto de datos de la gráfica de barras
data = go.Bar(
    x=dfr2.rating,
    y=dfr2.conteo,
    text=dfr2.conteo,
    textposition="outside"
)

# Crear el diseño de la gráfica
layout = go.Layout(
    title="Conteo de calificaciones",
    xaxis={'title': 'Rating'},
    yaxis={'title': 'Count'}
)

# Crear la figura con los datos y el diseño
fig = go.Figure(data=data, layout=layout)

# Mostrar la figura
fig.show()

##Cantidad de calificaciones por usuario

In [18]:
# Calcular cuántas películas calificó cada usuario
rating_users = pd.read_sql('''SELECT "userId" as user_id,
                                     COUNT(*) as cnt_rat
                              FROM ratings
                              GROUP BY "userId"
                              ORDER BY cnt_rat ASC''', conn)

In [19]:
# Crear un histograma de la frecuencia del número de calificaciones por usuario
fig = px.histogram(rating_users, x='cnt_rat', title='Histograma de la frecuencia del número de calificaciones por usuario')
fig.show()

In [20]:
rating_users.describe()
### la mayoria de usarios tiene pocos libros calificados, pero los que más tiene tiene muchos

,user_id,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


**Cantidad promedio de calificaciones por usuario:** La cantidad promedio de calificaciones por usuario es de aproximadamente 165 para el primer conjunto de datos y 305.5 para el segundo conjunto.

**Desviación estándar: **La desviación estándar indica la dispersión de los datos en torno a la media. Un valor más alto indica mayor variabilidad en la cantidad de calificaciones por usuario.

**Número mínimo y máximo de calificaciones por usuario:** El número mínimo de calificaciones por usuario es 20 para el primer conjunto de datos y 1 para el segundo, lo que indica que hay usuarios que han calificado relativamente pocas películas. El número máximo de calificaciones por usuario es 2698 para el primer conjunto de datos y 610 para el segundo, lo que indica que hay usuarios que han calificado un gran número de películas.

**Percentiles:** Los percentiles muestran cómo se distribuyen los datos. Por ejemplo, el 25% de los usuarios en el primer conjunto de datos han calificado 35 películas o menos, mientras que el 25% de los usuarios en el segundo conjunto de datos han calificado 35 películas o menos. El 50% de los usuarios en ambos conjuntos de datos han calificado 70 películas o menos.

In [21]:

# Consulta SQL para filtrar usuarios con más de 50 calificaciones pero menos de 1000
query = '''
    SELECT userId as user_id,
           COUNT(*) as cnt_rat
    FROM ratings
    GROUP BY userId
    HAVING cnt_rat >= 50 AND cnt_rat <= 1000
    ORDER BY cnt_rat ASC
'''

# Ejecutar la consulta y cargar los resultados en un DataFrame
rating_users2 = pd.read_sql(query, conn)

# Imprimir el DataFrame
print(rating_users2)

     user_id  cnt_rat
0         31       50
1        124       50
2        170       50
3        237       50
4        457       50
..       ...      ...
368      177      904
369      298      939
370      603      943
371      307      975
372      182      977

[373 rows x 2 columns]


In [22]:
### ver distribucion después de filtros
rating_users2.describe()

,user_id,cnt_rat
count,373.000000,373.000000
mean,301.766756,201.863271
std,178.462796,187.219913
min,1.000000,50.000000
25%,141.000000,77.000000
50%,305.000000,129.000000
75%,455.000000,248.000000
max,608.000000,977.000000


Hay un total de 373 usuarios que tienen entre 50 y 1000 calificaciones.

El promedio de calificaciones por usuario es aproximadamente 201.86, lo que sugiere que en promedio, los usuarios han calificado alrededor de 201 películas.

La desviación estándar es de aproximadamente 187.22, lo que indica que la dispersión de la cantidad de calificaciones entre los usuarios es considerable.

El usuario con el menor número de calificaciones tiene 50 calificaciones, mientras que el usuario con el mayor número de calificaciones tiene 977. Esto muestra una amplia variabilidad en el número de calificaciones entre los usuarios.

Los cuartiles muestran que el 25% de los usuarios tienen menos de 77 calificaciones, el 50% tienen menos de 129 calificaciones y el 75% tienen menos de 248 calificaciones.

En general, la mayoría de los usuarios tienen una cantidad considerable de calificaciones, con una variabilidad significativa en la cantidad de calificaciones entre los usuarios.

In [23]:
### graficar distribución después de filtrar datos
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usario')
fig.show()

In [26]:
#### verificar cuantas calificaciones tiene cada película
rating_movies = pd.read_sql('''SELECT movieId,
                                     COUNT(*) AS cnt_rat
                              FROM ratings
                              GROUP BY movieId
                              ORDER BY cnt_rat DESC
                            ''', conn)

In [27]:
### analizar distribucion de calificaciones por película
rating_movies.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


In [28]:
### graficar distribución
fig  = px.histogram(rating_movies, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada película')
fig.show()

Estos resultados muestran que hay un total de 9724 películas en el conjunto de datos. La columna "cnt_rat" indica el número de calificaciones que ha recibido cada película.

El promedio de calificaciones por película es aproximadamente 10.37.

La desviación estándar es alta, lo que indica que la cantidad de calificaciones por película está bastante dispersa.

La película con el menor número de calificaciones tiene solo una, mientras que la película con el mayor número de calificaciones tiene 329.

El 25% de las películas tienen una calificación o menos, el 50% tienen tres o menos, y el 75% tienen nueve o menos.

Hay una variabilidad considerable en la cantidad de calificaciones que reciben las películas, desde muy pocas hasta un número bastante alto.

In [29]:
####Filtrar películas que no tengan más de 50 calificaciones y usuarios que no tengan más de 10 películas calificados
rating_movies2 = pd.read_sql('''
    SELECT movieId, COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY movieId
    HAVING cnt_rat >= 50
    ORDER BY cnt_rat DESC
''', conn)

In [31]:
rating_movies2.describe()
fig  = px.histogram(rating_movies2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada películas')
fig.show()

# Cerrar la conexión
conn.close()